In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrs = {1: 'first', 2: 'second', 3: 'third', 4: 'fourth'}

qtrsrm = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}

In [ ]:
s = ['A939RC', 'A939RX']

df = nipa_df(retrieve_table('T70100')['Data'], s)
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1])  * df['A939RC'].iloc[-1]

In [ ]:
cd_gdp = (nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
          .sort_index().iloc[-1].values[0])

rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX']).sort_index()
rgdp_cd = rgdp / rgdp.iloc[-1] * cd_gdp

In [ ]:
year = df.index[-1].year
q = df.index[-1].quarter
text_date = f'{year} Q{q}'
date = df.index[-1].strftime('%Y-%m-%d')
gdp_val = int(rgdp_cd.iloc[-1].values[0] / 1000)
gdp_prv_val = int(rgdp_cd.iloc[0].values[0] / 1000)
gdppc_val = int(df.iloc[-1].loc['value'])
gdppc_prv_val = int(df.iloc[0].loc['value'])

f = text_dir.joinpath('gdp.txt')

In [ ]:
text = [r'\begin{minipage}{0.61\textwidth}',
f'\small This analysis of the United States economy begins with the most popular measure of economic activity, Gross Domestic Product (GDP). According to the Bureau of Economic Analysis, GDP--the seasonally-adjusted annualized value of goods and services produced in the US--was \${gdp_val:,} billion in the {qtrs[q]} quarter of {year}, compared to an inflation-adjusted equivalent of \${gdp_prv_val:,} billion in the first quarter of 1989.\\\\',
r'',
r'The US population is growing by about sixth-tenths of a percent per year. GDP per capita (see {\color{blue}\textbf{---}} ), adjusted for inflation to ' + text_date + r' dollars, has increased to ' + f'\${gdppc_val:,} in {text_date} from \${gdppc_prv_val:,} in 1989 Q1. \\\\',
r'\end{minipage}\hfill',
r'\begin{minipage}{0.34\textwidth}',
r'\noindent \normalsize \textbf{GDP per capita}\\',
r'\footnotesize{\textit{in ' + text_date + r' dollars}}\\ ',
r'\noindent \begin{tikzpicture}',
r'    \begin{axis}[\dateaxisticks clip=false, height=4.2cm, width=5.8cm,',
r'            ytick={40000,50000,60000}, yticklabels={40, 50, \$60k},',
r'            xtick={{1989-01-01}, {2000-01-01}, {2010-01-01}, {2019-04-01}},',
r'        minor xtick={}, ',
r"        xticklabels={`89, `00, `10, `19 Q2}, enlarge y limits={lower, 0.3}, ",
r'        enlarge x limits={0.05}]',
r'    \stdline{blue}{date}{value}{data/gdppc.csv}',
r'    \node[label={90:{\$'+ f'{gdppc_val:,}' + r'}}, circle, blue, fill, inner sep=1.5pt] at ',
f'        (axis cs:{date},{gdppc_val})' + r' {};',
r'    \end{axis}',
r'\end{tikzpicture}\\',
r'\footnotesize{Source: Bureau of Economic Analysis} \\',
r'\end{minipage}']

In [ ]:
with f.open('w') as wf:
    for t in text:
        print(t)
        wf.write(t + '\n')

In [ ]:
f = text_dir.joinpath('gdp_gr.txt')

gdp_gr = nipa_df(retrieve_table('T10502')['Data'], ['A191RL']).sort_index()
year = gdp_gr.index[-1].year
year_p = gdp_gr.index[-2].year
q = gdp_gr.index[-1].quarter
qp = gdp_gr.index[-2].quarter
text_date = f'{year} Q{q}'
date = gdp_gr.index[-1].strftime('%Y-%m-%d')
gr = gdp_gr.iloc[-1].values[0]
if gr >= 0:
    inc_dec = 'increased'
else:
    inc_dec = 'decreased'
    
gr_p = gdp_gr.iloc[-2].values[0]
if gr_p >= 0:
    inc_dec_p = 'an increase'
else:
    inc_dec_p = 'a decrease'
    
gr_3 = round(gdp_gr.iloc[-13:].values.mean(), 1)
gr_10 = round(gdp_gr.iloc[-41:].values.mean(), 1)
gr_30 = round(gdp_gr.iloc[-121:].values.mean(), 1)

In [ ]:
gdpg = [f'{inc_dec} at an annual rate of {gr} percent during the {qtrs[q]} quarter of {year}, compared to {inc_dec_p} of {gr_p} percent in the {qtrs[qp]} quarter of {year_p}. Quarterly growth has averaged {gr_3} percent over the past three years, {gr_10} percent over the past 10 years, and {gr_30} percent over the past 30 years. ']

In [ ]:
f = text_dir.joinpath('gdp_gr.txt')
with f.open('w') as wf:
    for t in gdpg:
        print(t)
        wf.write(t + '\n')

### Simple text for pce growth

In [6]:
s = pd.read_csv(data_dir / 'pcegrowth.csv', parse_dates=['date']).set_index('date')['value2']
d = series_info(s)

In [10]:
textval = f'Consumer spending is also reported on a monthly basis. Inflation- and population-adjusted consumer spending increased by {d["val_latest"]:.1f} percent over the year ending {d["date_latest"].strftime("%B %Y")}, compared to an increase of {d["val_year_ago"]:.1f} percent over the year ending {d["date_year_ago"].strftime("%B %Y")}. '

In [11]:
f = text_dir.joinpath('pcegrowth.txt')
with f.open('w') as wf:
    wf.write(textval)

### Simple text for cpi

In [25]:
s = pd.read_csv(data_dir / 'cpi.csv')#, parse_dates=['date']).set_index('date')

In [26]:
date = pd.to_datetime(s['date'].iloc[-1]).strftime('%B %Y')
core = s['value2'].iloc[-1]
allitems = s['value'].iloc[-1]

In [27]:
text = f'Consumer prices increased by {allitems:.1f} percent over the year ending {date}, according to the CPI for all urban consumers. Core inflation, which does not include the more volatile food and energy prices, was {core:.1f} percent.'

In [28]:
f = text_dir.joinpath('cpi_main.txt')
with f.open('w') as wf:
    wf.write(text)

### Simple text for epop

In [ ]:
s = pd.read_csv(data_dir / 'epop.csv').iloc[-1]
date = pd.to_datetime(s.date).strftime('%Y-%m')
value = s.PA_EPOP

txt = f'{date}:\\\{value}\%'

f = text_dir.joinpath('epop.txt')
with f.open('w') as wf:
    wf.write(txt)

### Body text for epop

In [2]:
s = pd.read_csv(data_dir / 'epop.csv', parse_dates=['date']).set_index('date')['PA_EPOP']
d = series_info(s)

In [ ]:
text = f'In {d["date_latest"].strftime("%B %Y")}, {d["val_latest"]}\%'

if (d['days_since_match'] > 725) | (d['days_since_match'] == 0):
    text2 = d['last_matched']
else:
    text2 = f'compared to {d["val_prev"]} in {d["date_prev"].strftime("%B %Y")}'
    
if d['change_12m'] > 0:
    direction = 'increased'
    value = d['change_12m']
    label = f'Over the past year, the age 25-54 employment rate has {direction} by {value:.1f} percentage points.'
elif d['change_12m'] < 0:
    direction = 'fallen'
    value = abs(d['change_12m'])
    label = f'Over the past year, the age 25-54 employment rate has {direction} by {value:.1f} percentage points.'
else:
    label = 'The age 25-54 employment rate is unchanged over the past year. '
    
pop = 126277
diff1 = d['late90s'] - d['val_latest']
diff = (d['late90s'] - d['val_latest']) / 100 * pop

if diff > 999:
    diff_text = f'{round(diff / 1000, 1)} million'
else: 
    diff_text = f'{round(diff, -1)} thousand'
    
label2 = f'The current age 25-54 employment rate is {diff1:.1f} percentage points (equivalent to {diff_text} workers) below the average during 1998--99, a period with a particularly tight labor market. '

In [ ]:
textval = f'{text} of 25-54 years olds were employed, {text2}. {label} {label2}'
print(textval)

In [ ]:
f = text_dir.joinpath('epop_text.txt')
with f.open('w') as wf:
    wf.write(textval)